# RobotInterfaceの使い方

In [1]:
exec(open('/choreonoid_ws/install/share/irsl_choreonoid/sample/irsl_import.py').read())

## インターフェースの初期化

In [2]:
ri = RobotInterface('/userdir/robotinterface.yaml', connection_wait=3.0)

## インターフェースのロボットモデルを使う
- クラスを登録しておくと良い
- クラス化が無ければ、都度register(end-effector, pose)する

関節全体を'arm'として登録している。'joint2'は終端のジョイントになります。

robotオブジェクトは「ロボットモデルの使い方」で説明したものと同じです。

In [ ]:
robot = ri.getRobotModel()
robot.registerEndEffector('arm', 'joint2', joint_list=m.jointNames)

## 関節の情報

### jointGroupについて

グループの確認

In [ ]:
ri.jointGroupList

<coordinates[0x562d2b743410] 0 0 0.784 / 0 0 0 1 >

グループの名前

In [ ]:
ri.jointGroupNames

デフォルトグループの関節

In [ ]:
ri.getJointGroup('default').jointNames

In [ ]:
ri.getJointGroup('default') is ri.jointGroupList[0]

### 関節に関するセンシング情報

センシングした関節角度

In [ ]:
ri.actualAngleVector

参照値（コマンドで送っている値、現状ではassemblerの実機では取れない）

In [ ]:
ri.referenceAngleVector

速度

In [ ]:
ri.velocityVector

エフォート（トルク、力）

In [ ]:
ri.effortVector

## ロボットへ関節角度を送る

### モデルの姿勢を実機へ送る

モデルへ角度セット、名前指定 (表示が変わる)

In [ ]:
robot.setJointMap({'joint2': -0.4})

角度を送る

```ri.sendAngleVector(<angle-vector>, <動作時間[秒]>)```

In [ ]:
ri.sendAngleVector(robot.angleVector(), 4.0)

動作終了を待つ

```ri.waitUntilFinish(<タイムアウト[秒]>)```

In [ ]:
ri.waitUntilFinish(5.0)

モデルへ角度セット、ベクトル指定(表示が変わる)

In [ ]:
robot.angleVector(nap([0.8, 0.8]))

角度を送る

In [ ]:
ri.sendAngleVector(robot.angleVector(), 10.0)

動作終了を待つ

In [ ]:
ri.waitUntilFinish(11.0)

### IKを使って姿勢を変える

In [ ]:
cds = robot.arm.endEffector

In [ ]:
cds.translate(npa([0.04, 0, 0]))

In [ ]:
robot.arm.inverseKinematics(cds, constraint='xyz')

In [ ]:
ri.sendAngleVector(robot.angleVector(), 2.0);

In [ ]:
ri.waitUntilFinish(3.0)

## 台車を動かす

台車を移動させる

```ri.move_velocity(<x軸方向速度>, <y軸方向速度>, <z軸回転角速度>)```

In [ ]:
ri.move_velocity(0.1, 0, 0)

waitする。現状はスリープしている。

In [ ]:
r.waitFinishMoving(2.0)

動作停止

In [ ]:
ri.stop()

## センサーデータの使い方

### センサデバイスの名前

In [ ]:
ri.deviceNames

### データを受け取る
- バッファにデータがあればデータを返す
- バッファにデータが無ければNoneが返る
- clearするとバッファからデータを除く

（前提: データを受け取るとバッファに入る）

In [ ]:
data = ri.data('', clear=False)
data

### データが来るまで待つ
- バッファにデータがあればデータを返す
- バッファにデータが無ければtimeoutするまで待つ
  - データが来れば返す、来なければNoneが返る
- clearするとバッファからデータを除く

In [ ]:
data = ri.waitData('', timeout=1.0, clear=False)
data

### 次のデータが来るまで待つ
- バッファのデータと無関係に、データが来るのをtimeoutするまで待つ
  - データが来れば返す、来なければNoneが返る
- clearするとバッファからデータを除く  


In [ ]:
data = ri.waitNextData('', timeout=1.0, clear=False)
data

## ループの作り方

In [ ]:
import rospy
while rospy.ok():
    ## <programs>
    rospy.sleep(0.1)
